# Import data

In [143]:
import pandas
base_data_url = "../datasets/project/road safety/road-safety-data-{}"

In [144]:
accidents_16 = pandas.read_csv(base_data_url.format("accidents-2016.csv"))
accidents_15 = pandas.read_csv(base_data_url.format("accidents-2015.csv"))
accidents_14 = pandas.read_csv(base_data_url.format("accidents-2005-2014.csv"))

In [145]:
accidents = pandas.DataFrame()
accidents = pandas.concat([accidents_16, accidents_15, accidents_14], axis=0)

In [146]:
# accidents

# Investigate correlations
1. Are serious or fatal accidents more likely in bad weather?
2. Are serious or fatal accidents more likely during the night?
3. Do serious or fatal accidents have, on average, a longer distance from a streetlight?
4. Do any times of day, days of week, or weeks of year have more serious or fatal accidents?

# 1: Bad weather
## Plotting

In [147]:
# Replace the weather with a number: 1 for bad weather, 0 for good
replacement_list = {'Fine + high winds': 1,
  'Fine no high winds': 0,
  'Fog or mist': 1,
  'Other': 0,
  'Raining + high winds': 1,
  'Raining no high winds': 1,
  'Snowing + high winds': 1,
  'Snowing no high winds': 1}

In [148]:
severities = ["Serious", "Slight", "Fatal"]

In [149]:
w_dict = {}
for severity in severities:
    w_dict[severity] = {}
    for weather_type in replacement_list.keys():
        matches = len(accidents[(accidents["Weather_Conditions"] == weather_type) &
                               (accidents["Accident_Severity"] == severity)])
        w_dict[severity][weather_type] = matches

In [150]:
# Plot results
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,20)

d_serious = w_dict["Serious"]
d_slight = w_dict["Slight"]
d_fatal = w_dict["Fatal"]


In [151]:
def plot_wtype(wtype, title):
    plt.figure()
    plt.title = title
    plt.bar(range(len(wtype)), list(wtype.values()), align='center')
    plt.xticks(range(len(wtype)), list(wtype.keys()))
    plt.show()

In [152]:
# plot_wtype(d_fatal, "Fatal")

## Hypothesis testing
$H_0$: Prop(bad, (serious|fatal)) = Prop(bad, slight)


$H_1$: Prop(bad, (serious|fatal)) > Prop(bad, slight)

For a p-value of 0.05.

In [211]:
# Replace the weather
accidents["Weather_Bool"] = accidents["Weather_Conditions"].map(replacement_list)

In [212]:
slight_list = list(accidents[
    (accidents["Accident_Severity"] == "Slight")]
                   .Weather_Bool)

In [213]:
bad_list = list(accidents[
    (accidents["Accident_Severity"] == ("Serious" or "Fatal"))]
                   .Weather_Bool)

In [214]:
# Actually do the hypothesis test
from scipy.stats import ttest_ind
ttest_ind(slight_list, bad_list)

Ttest_indResult(statistic=1.1418463195447863, pvalue=0.25362583312719567)

p > 0.05, so the result is not significant and we accept the null hypothesis.

# 2: Night-time vs. daytime
$H_0$: Prop(serious, nighttime) = Prop(slight, nighttime)

$H_1$: Prop(serious, nighttime) > Prop(slight, nighttime)

p = 0.05.

In [215]:
accidents = pandas.concat([accidents, accidents_15, accidents_14], axis=0)

In [216]:
import datetime
from astral import Astral
city_name = 'Edinburgh'
a = Astral()
a.solar_depression = 'civil'
city = a[city_name]

def sun_at_day(day):
    sun = city.sun(date = day, local=False)
    return sun

def is_day(x):
    time_x = datetime.datetime.strptime(x + "+0000", '%d/%m/%Y %H:%M%z')
    sun = sun_at_day(time_x)
    
    
    if (sun['sunrise'] < time_x < sun['sunset']):
        return 0
    return 1

In [217]:
bad_accidents = accidents[(accidents["Accident_Severity"] == ("Serious" or "Fatal"))]
slight_accidents = accidents[(accidents["Accident_Severity"] == "Slight")]

In [218]:
bad_dt = bad_accidents.Date + " " + bad_accidents.Time
slight_dt = slight_accidents.Date + " " + slight_accidents.Time

In [219]:
bad_dt = bad_dt.map(lambda x: is_day(x))
slight_dt = slight_dt.map(lambda x: is_day(x))

In [220]:
from scipy.stats import ttest_ind
ttest_ind(bad_dt, slight_dt)

Ttest_indResult(statistic=2.2383240618397853, pvalue=0.025244367725947698)

p < 0.05 so we reject the null hypothesis: the proportion of bad accidents during the night is greater than slight accidents during the night.

# 2.5: Darkness vs daytime


In [221]:
accidents = pandas.DataFrame()
accidents = pandas.concat([accidents_16, accidents_15, accidents_14], axis=0)

lights = {"Darkness - lights lit": 1,
         "Darkness - no lighting": 0,
         "Daylight": 1,
         "Darkness - lighting unknown": 0,
         "Darkness - lights unlit": 0}
accidents["Light_Bool"] = accidents["Light_Conditions"].map(lights)

In [222]:

bad_accidents = accidents[(accidents["Accident_Severity"] == ("Serious" or "Fatal"))]
slight_accidents = accidents[(accidents["Accident_Severity"] == "Slight")]


In [223]:
bad_l = list(bad_accidents["Light_Bool"])
slight_l = list(slight_accidents["Light_Bool"])

In [224]:
ttest_ind(bad_l, slight_l)

Ttest_indResult(statistic=-1.244340868497855, pvalue=0.21348926238999366)

# Pretty plots

In [21]:
import folium

colors = {'Slight': 'green', 'Serious': 'orange', 'Fatal': 'red'}

map_acc = folium.Map(location=[56.4620, -2.9707], zoom_start=12)


In [22]:
accidents = pandas.DataFrame()
# Something is up with _15, _14, so use only _16...
#accidents = pandas.concat([accidents_16, accidents_15, accidents_14], axis=0)
accidents = accidents_16

In [23]:
accidents.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                               radius=10, 
                                               fill_color=colors[row["Accident_Severity"]],
                                               fill = True,
                                               stroke=False,
                                               fill_opacity=0.5,
                                              popup = row["Accident_Index"])
                                             .add_to(map_acc), axis=1)

map_acc

In [227]:
import numpy
cameras = pandas.read_csv("../datasets/project/cameras_wgs.csv", delimiter='\t')
# Need to filter out nans here
cameras = cameras[numpy.isfinite(cameras['Lat'])]

In [228]:
cameras_coords = pandas.concat([cameras.Lat, cameras.Lon], axis=1)

In [230]:
def notnan(x):
    try: 
        int(x)
        return True
    except:
        return False

In [231]:
from scipy.spatial import Voronoi, voronoi_plot_2d
cameras_list = [list(x) for x in cameras_coords.values if notnan(x[0])]

vor = Voronoi(cameras_list) 

In [250]:
%%capture
# Now actually plot with folium
map_cam = folium.Map(location=[56.4620, -2.9707], zoom_start=12)

cameras.apply(lambda row:folium.Circle(location=[row["Lat"], row["Lon"]], 
                                       radius=10, 
                                       color='black',
                                       popup = row["Camera Location"])
                                      .add_to(map_cam), axis=1)

In [251]:
# Add voronois
# http://comet.lehman.cuny.edu/owen/teaching/datasci/makeVor.py

from geojson import FeatureCollection, Feature, Polygon

vorJSON = open('libVor.json', 'w')
point_voronoi_list = []
feature_list = []
for region in range(len(vor.regions)-1):
    vertex_list = []
    for x in vor.regions[region]:
        #Not sure how to map the "infinite" point, so, leave off those regions for now:
        if x == -1:
            break;
        else:
            #Get the vertex out of the list, and flip the order for folium:
            vertex = vor.vertices[x]
            vertex = (vertex[1], vertex[0])
        vertex_list.append(vertex)
    #Save the vertex list as a polygon and then add to the feature_list:
    polygon = Polygon([vertex_list])
    feature = Feature(geometry=polygon, properties={})
    feature_list.append(feature)

#Write the features to the new file:
feature_collection = FeatureCollection(feature_list)
print (feature_collection, file=vorJSON)
vorJSON.close()


folium.GeoJson(
    "libVor.json",
    name='geojson'
).add_to(map_cam)

In [252]:
map_cam